In [1]:
%matplotlib widget
import matplotlib.pyplot as plt
import numpy as np
import lightkurve as lk
from astropy.table import Table
import scipy.signal as signal
import scipy.optimize as optimize
from tqdm import tqdm
from modules import *

plt.style.use('seaborn-v0_8-darkgrid')

TIC = 'TIC 219107776'

In [ ]:
lightcurves = get_lightcurves(TIC, author = 'SPOC', cadence='short', use_till=2)
lightcurve = lightcurves[0]

In [ ]:
periodogram = lightcurve.to_periodogram(maximum_frequency = 30)

peak_width = 1 / ( (lightcurve.time.btjd[-1] - lightcurve.time.btjd[0]) * 2)
print(peak_width)
max_power = periodogram.max_power.value
optimized_parameters_global = []

colors = ['red', 'green', 'purple', 'orange', 'yellow', 'pink', 'brown', 'magenta', 'cyan']
def color_change():
    c = colors.pop(0)
    colors.append(c)
    yield c

frequencies = periodogram.frequency.value
power = periodogram.power.value
f = np.linspace(frequencies.min(), frequencies.max(), 1000000)
p = spline(frequencies, power, k = 3)(f)

peaks, peakdict = signal.find_peaks(p, height = max_power/10)
peaks_df = pd.DataFrame({'frequencies': f[peaks], 'power': p[peaks], 'peak_index': peaks})

fi = go.FigureWidget([
    go.Scatter(x=f[peaks], y=p[peaks], mode = 'markers', name = 'Peaks', marker = {
        'symbol': 'x',
        'size': 7,
    }),
    go.Scatter(x=frequencies, y=power, mode='lines', name = 'Periodogram', line = dict(color='black')),
], layout = go.Layout(title = 'Periodogram Peaks', xaxis_title = 'Frequency [1/d]', yaxis_title = 'Power'))

scatter = fi.data[0]
fi.layout.hovermode = 'closest'

def update_point(trace, points, selector):
    for i in points.point_inds:
        initial_guess = [peaks_df['power'][i], peaks_df['frequencies'][i], peak_width]
        optimized_parameters, covariance = optimize.curve_fit(gaussian, frequencies, power, p0=initial_guess)
        amp, cen, wid = optimized_parameters

        fitted_gaussian = gaussian(frequencies, amp, cen, wid)
        initial_gaussian = gaussian(frequencies, *initial_guess)

        fi.add_trace(go.Scatter(x=frequencies, y=fitted_gaussian, mode='lines', name='Fitted Gaussian', line=dict(color=next(color_change()))))
        fi.add_trace(go.Scatter(x=frequencies, y=initial_gaussian, mode='lines', name='Initial Gaussian', line=dict(color=next(color_change()))))
        optimized_parameters_global.append(optimized_parameters)
        print(optimized_parameters_global[-1])
    
scatter.on_click(update_point)
fi


In [ ]:
try:
    fitted_frequency = optimized_parameters_global[-1][1]
    fitted_period = 1/fitted_frequency
except:
    raise ValueError('No peaks clicked yet, please click on a peak to fit a gaussian to it.')

print(f'Fitted Frequency: {fitted_frequency}')
print(f'Fitted Period: {fitted_period}')

In [ ]:
#IGNORE FOR NOW, Just using for syntax reference
lightcurve.fold(fitted_period).bin(time_bin_size = 0.0005).scatter()